<a href="https://colab.research.google.com/github/Syafunn/repo3/blob/main/percobaan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import os

os.makedirs("data/asl_data", exist_ok=True)


In [2]:
!mkdir -p data/asl_data

In [3]:
from google.colab import files
uploaded = files.upload()

Saving sign_mnist_test.csv to sign_mnist_test.csv
Saving sign_mnist_train.csv to sign_mnist_train.csv


In [4]:

import shutil

shutil.move("sign_mnist_train.csv", "data/asl_data/sign_mnist_train.csv")
shutil.move("sign_mnist_test.csv", "data/asl_data/sign_mnist_valid.csv")

'data/asl_data/sign_mnist_valid.csv'

In [5]:
import tensorflow.keras as keras
import pandas as pd

# Load in our data from CSV files
train_df = pd.read_csv("data/asl_data/sign_mnist_train.csv")
valid_df = pd.read_csv("data/asl_data/sign_mnist_valid.csv")

# Separate out our target values
y_train = train_df['label']
y_valid = valid_df['label']
del train_df['label']
del valid_df['label']

# Separate out our image vectors
x_train = train_df.values
x_valid = valid_df.values

# Otomatis hitung jumlah kelas dari data
num_classes = max(y_train.max(), y_valid.max()) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

# Normalize our image data
x_train = x_train / 255
x_valid = x_valid / 255

In [6]:
x_train.shape, x_valid.shape

((27455, 784), (7172, 784))

In [7]:
x_train = x_train.reshape(-1,28,28,1)
x_valid = x_valid.reshape(-1,28,28,1)

In [8]:

x_train.shape

(27455, 28, 28, 1)

In [9]:
x_valid.shape

(7172, 28, 28, 1)

In [10]:

x_train.shape, x_valid.shape

((27455, 28, 28, 1), (7172, 28, 28, 1))

In [11]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)

model = Sequential()
model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu",
                 input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(25, (3, 3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Flatten())
model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(units=num_classes, activation="softmax"))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 75)     │           750 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 28, 28, 75)     │           300 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 75)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 50)     │        33,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 50)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 14, 14, 50)     │           200 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 50)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 25)       │        11,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 7, 7, 25)       │           100 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 25)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       205,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 25)             │        12,825 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 264,562 (1.01 MB)

 Trainable params: 264,262 (1.01 MB)

 Non-trainable params: 300 (1.17 KB)

In [13]:
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])

In [14]:
model.fit(x_train, y_train, epochs=20, verbose=1, validation_data=(x_valid, y_valid))


Epoch 1/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 120s 135ms/step - accuracy: 0.7572 - loss: 0.8267 - val_accuracy: 0.7927 - val_loss: 0.6909
Epoch 2/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 143s 137ms/step - accuracy: 0.9922 - loss: 0.0245 - val_accuracy: 0.8621 - val_loss: 0.5288
Epoch 3/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 136s 131ms/step - accuracy: 0.9958 - loss: 0.0134 - val_accuracy: 0.8682 - val_loss: 0.5121
Epoch 4/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 113s 132ms/step - accuracy: 0.9978 - loss: 0.0060 - val_accuracy: 0.8318 - val_loss: 0.9162
Epoch 5/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 141s 131ms/step - accuracy: 0.9979 - loss: 0.0061 - val_accuracy: 0.9576 - val_loss: 0.1818
Epoch 6/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 141s 130ms/step - accuracy: 0.9989 - loss: 0.0030 - val_accuracy: 0.9568 - val_loss: 0.1809
Epoch 7/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 137s 124ms/step - accuracy: 0.9991 - loss: 0.0035 - val_accuracy: 0.9328 - val_loss: 0.3842
Epoch 8/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 114s 133ms/step - accuracy: 0.9996 -

In [15]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}